In [1]:
import pandas as pd
import requests
import json
from datetime import datetime, date, timedelta
from pytz import timezone

In [7]:
# read the city dataframe to get the details of all cities where we need airport data
# use the latitude & longitude to provide argument in function call

city_data = pd.read_csv("city_data.csv")
city_data.drop(columns = ["Unnamed: 0"], inplace = True)
#city_data

In [8]:
# convert the latitude & longitude columns to lists to be used as input of a function call

lat = city_data["city_latitude"].to_list()
#lat 

lon = city_data["city_longitude"].to_list()
#lon

In [ ]:
# example call to get airports from location by lat/lon within 50km

url = "https://aerodatabox.p.rapidapi.com/airports/search/location/50.110924/8.682127/km/50/16"

querystring = {"withFlightInfoOnly":"true"}

headers = {
	"X-RapidAPI-Key": "Use Your API key here",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

#print(response.json())

In [ ]:
# convert json to a normalised table

airports = pd.json_normalize(response.json()['items'])

# airports

In [11]:
# test the call as a function

def icao_codes(latitude,longitude):
    
    list_for_df = []
    
    for i in range(len(latitude)):
        
        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitude[i]}/{longitude[i]}/km/50/16"
        querystring = {'withFlightInfoOnly': 'true'}
        headers = {
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
            "X-RapidAPI-Key": "Use Your API key here",        
        }
        response = requests.request('GET', url, headers = headers, params=querystring)
        
        list_for_df.append(pd.json_normalize(response.json()['items']))

    return pd.concat(list_for_df, ignore_index=True)
                                   

In [12]:
icao_codes([50.0264],[8.543129])

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon
0,EDDF,FRA,Frankfurt-am-Main,Frankfurt-am-Main,Frankfurt-am-Main,DE,50.0264,8.543129


In [20]:
# rewrite the code above as a function so it can be reused again elsewhere, include exception test/rule

def icao_airport_code(latitude, longitude):
    airport_list = []
    # check the length of the latitude and longitude lists to make sure they are equal
    assert len(latitude) == len(longitude)
    # set the API call to get airport data within 50km of the lat and lon being input and show 10 results
    # using previous call example
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location/51.511142/-0.103869/km/50/10"
    querystring = {"withFlightInfoOnly":"true"}
    headers = {
        "X-RapidAPI-Key": "Use Your API key here",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }
    test = requests.request("GET", url, headers = headers, params = querystring)
    if test.status_code >= 200 and test.status_code <= 299:                      # check for good connection else -1?? 
        for i in range(len(latitude)):
            url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitude[i]}/{longitude[i]}/km/50/10"
            querystring = {"withFlightInfoOnly":"true"}
            headers = {
                "X-RapidAPI-Key": "Use Your API key here", 
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
            response = requests.request("GET", url, headers = headers, params = querystring)
            airport_df = pd.json_normalize(response.json()["items"])
            airport_list.append(airport_df)
    else:
        return -1
    airports_df = pd.concat(airport_list, ignore_index = True)
    return airports_df

In [13]:
# test function using the earlier defined lat & lon lists from the imported city_data

airport_data = icao_airport_code(lat, lon)
#airport_data

In [11]:
# remove all air bases from the result

airport_data = airport_data[~airport_data.name.str.contains("Air Base", case = False)]
#airport_data

In [10]:
# remove all duplicate icao

airport_data.drop_duplicates(subset = "icao", inplace = True)
#airport_data

In [17]:
# drop unwanted columns

airport_data.drop(columns = ["shortName", "localCode"], inplace = True)
#airport_data

In [ ]:
# rename columns with more intuitive names and replace with underscore dots to avoid problems in MySQL

airport_data.rename(columns = {"icao": "airport_icao",
                               "iata":"airport_iata",
                               "name": "airport_name",
                               "municipalityName": "municipality_name",
                               "countryCode": "country_code",
                               "location.lat": "airport_latitude",
                               "location.lon": "airport_longitude"},
                    inplace = True)
#airport_data

In [ ]:
# edit entries in the municipality_name column to match the city name for a merge operation later
airport_data.at[5, "municipality_name"] = "Düsseldorf"
airport_data.at[6, "municipality_name"] = "Frankfurt am Main"
#airport_data

In [110]:
# merge the airport data with the city data to add a city_id column to airport data

airport_data = airport_data.merge(city_data[["city_id", "city"]], how = "left", left_on = "municipality_name", right_on = "city")

#airport_data

In [111]:
airport_data.drop(columns = ["city"], inplace = True) # as repeat of municipality_name

In [112]:
airport_data.reset_index(drop = True, inplace = True)
airport_data

,icao,iata,airport_name,municipality_name,country_code,airport_latitude,airport_longitude,city_id
0,EDDT,TXL,"Berlin, Berlin-Tegel",Berlin,DE,52.55970,13.287699,Q64
1,EDDB,BER,"Berlin, Berlin Brandenburg",Berlin,DE,52.35139,13.493889,Q64
2,EDDH,HAM,Hamburg,Hamburg,DE,53.63040,9.988229,Q1055
3,EDDM,MUC,Munich,Munich,DE,48.35380,11.786100,Q1726
4,EDDK,CGN,"Cologne, Cologne Bonn",Cologne,DE,50.86590,7.142739,Q365
5,EDDL,DUS,"Duesseldorf, Düsseldorf",Düsseldorf,DE,51.28950,6.766779,Q1718
6,EDDF,FRA,Frankfurt-am-Main,Frankfurt am Main,DE,50.02640,8.543129,Q1794
7,EDDS,STR,Stuttgart,Stuttgart,DE,48.68990,9.221960,Q1022
8,EDLW,DTM,Dortmund,Dortmund,DE,51.51830,7.612239,Q1295
9,EDDP,LEJ,"Leipzig, Leipzig Halle",Leipzig,DE,51.43240,12.241600,Q2079


In [113]:
airport_data.to_csv("airport_data.csv")